<a href="https://colab.research.google.com/github/11aleksey11/connector_neo4j/blob/main/test_neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j
import json
import pandas as pd
from neo4j import GraphDatabase
from fastapi import FastAPI


*Получаем url для запроса к файлу *.csv и создаём dataframe*

In [ ]:
base_url = 'https://drive.google.com/file/d/1Q7r_rwyB91f1eiVDvjPfUsITzhxPV08M/view?usp=sharing'
url = 'https://drive.google.com/uc?id=' + base_url.split('/')[-2]
df = pd.read_csv(url)

lst_person = []
rows = ''
for i in df.iteritems():
    rows = i[1]
for row in rows:
    lst_person.append({'pers1': row.split(';')[1], 'pers2': row.split(';')[2], 'Event': row.split(';')[0]})


# создаём подключение к neo4j

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_request(self, lst):
        with self.driver.session(database="neo4j") as session:
            result = session.execute_write(
                self._create_and_return_friendship, lst)

    @staticmethod
    def _create_and_return_friendship(tx, lst):
        set_person = set()
        query = ''
        count = 0
        for dct in lst:
            if dct['pers2'] not in set_person:
                query = ', '.join([f'({dct["pers2"].replace(" ", "_")}:Person {{ name: "{dct["pers2"]}"}})', query])
                set_person.add(dct['pers2'])
            if dct['pers1'] not in set_person:
                query = ', '.join([f'({dct["pers1"].replace(" ", "_")}:Person {{ name: "{dct["pers1"]}"}})', query])
                set_person.add(dct['pers1'])
            if dct['Event'] not in set_person:
                query = ', '.join([f'(Event_{str(dct["Event"])}:Event {{ name: "{dct["Event"]}"}})', query])
                set_person.add(dct['Event'])
            make_relation1 = f'({dct["pers1"].replace(" ", "_")})-[:EVENT]->(Event_{str(dct["Event"])})'
            make_relation2 = f'({dct["pers2"].replace(" ", "_")})-[:EVENT]->(Event_{str(dct["Event"])})'
            query = ', '.join([query, make_relation1, make_relation2])
        result = tx.run('create ' + query.replace(', ,', ', '))
        return [result]

    def get_request(self, lst):
        with self.driver.session(database="neo4j") as session:
            result = session.execute_read(
                self._create_and_return_get_request, lst)

    app = FastAPI()

    @app.post("/match")
    def return_name_json(self, name='Галчевская Карина Владимировна'):
        session = self.driver.session()
        query = f'match (n:Person {{name:"{name}"}})-[a:EVENT]-(s:Event) return n, a, s'
        result = json.dumps(session.run(query).data())
        decoding_result = json.loads(result)
        return decoding_result


if __name__ == "__main__":
    uri = "neo4j+s://e2300488.databases.neo4j.io"
    user = "neo4j"
    password = "X-3JMVu5spbZqt-J_Ov88ceI8vbrdFVDLK9i4MrovB0"
    app = App(uri, user, password)
    #app.create_request(lst_person)
    js_result = app.return_name_json()
    app.close()